<a href="https://colab.research.google.com/github/cyaag/Enterprise-Scale/blob/main/Call_Volume_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#2.2 Exercise- Solution Call Volume Forecasting

In [ ]:
# If running in Azure ML Notebooks, the workspace is pre-configured.
from azureml.core import Workspace, Dataset, Run
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Connect to your Azure ML workspace
ws = Workspace.from_config()  # Use the config.json in the project

In [ ]:
# Load data directly from the CSV file in the project folder
# The 'parse_dates' argument tells pandas to automatically convert date columns
df = pd.read_csv('data_call_center.csv', parse_dates=['occurrence_date', 'closure_date'])

print("Data loaded successfully. Here's a preview:")
df.head()

##Step 1:Preprocesss the Data for required Features

In [ ]:
# Create seasonality features from the 'occurrence_date'
df['day_of_week'] = df['occurrence_date'].dt.dayofweek  # Monday=0, Sunday=6
df['month'] = df['occurrence_date'].dt.month

# One-hot encode the 'channel' column to use it as a feature
df = pd.get_dummies(df, columns=['channel'], drop_first=True)

# Define the features (X) and the target variable (y)
features = ['day_of_week', 'month', 'promotion', 'downtime', 'channel_voice']
X = df[features]
y = df['call_volume']

print("\nFeatures for the model:")
X.head()

## Step 2:Train and Split the Data for Testing and Traininng

In [ ]:
# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTraining set size: {len(X_train)} rows")
print(f"Testing set size: {len(X_test)} rows")

## Step 3: Regression Model for Call Volume

In [ ]:
# Initialize and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
print(f"\nModel training complete.")
print(f"Mean Absolute Error on Test Data: {mae:.2f}")

##Visualize and Forecast

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(y_test.values, label='Actual Call Volume', marker='o')
plt.plot(y_pred, label='Predicted Call Volume', marker='x', linestyle='--')
plt.title('Call Volume Forecast vs. Actual')
plt.xlabel('Test Sample Index')
plt.ylabel('Call Volume')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Assume each agent can handle 40 calls per shift
calls_per_agent = 40

print("\n--- Staffing Recommendations for Next Day ---")

# --- Predict for a VOICE channel on a Wednesday with a promotion ---
next_day_voice = pd.DataFrame({
    'day_of_week': [2],  # Wednesday
    'month': [1],        # January
    'promotion': [1],    # Promotion is active
    'downtime': [0],     # No downtime
    'channel_voice': [1] # 1 for voice channel
})
predicted_calls_voice = model.predict(next_day_voice)[0]
required_agents_voice = int(np.ceil(predicted_calls_voice / calls_per_agent))

print(f"Channel: Voice | Predicted Calls: {predicted_calls_voice:.0f} | Recommended Agents: {required_agents_voice}")

# --- Predict for a CHAT channel on a Wednesday with a promotion ---
next_day_chat = pd.DataFrame({
    'day_of_week': [2],  # Wednesday
    'month': [1],        # January
    'promotion': [1],    # Promotion is active
    'downtime': [0],     # No downtime
    'channel_voice': [0] # 0 for chat channel
})
predicted_calls_chat = model.predict(next_day_chat)[0]
required_agents_chat = int(np.ceil(predicted_calls_chat / calls_per_agent))

print(f"Channel: Chat  | Predicted Calls: {predicted_calls_chat:.0f} | Recommended Agents: {required_agents_chat}")

In [ ]:
# Save the trained model to a file for later use or deployment
model_filename = 'call_volume_forecast_model.pkl'
joblib.dump(model, model_filename)

print(f"\nModel saved successfully as '{model_filename}'")